<a href="https://colab.research.google.com/github/hmyrcmn/FaceMaskDetectionYolo/blob/main/denemeMutfak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np

# Veri setinin yolu
temiz_yolu = '/content/clean'
kirli_yolu = '/content/dirty'

def load_images(path):
    images = []
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))
            images.append(img)
    return np.array(images)

temiz_goruntuler = load_images(temiz_yolu)
kirli_goruntuler = load_images(kirli_yolu)

# Etiketler oluştur
temiz_labels = np.zeros((temiz_goruntuler.shape[0], 1))
kirli_labels = np.ones((kirli_goruntuler.shape[0], 1))


In [2]:
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow as tf

def build_siamese_model(input_shape):
    input = Input(input_shape)

    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)

    model = Model(input, x)
    return model

def siamese_network(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)

    model = build_siamese_model(input_shape)

    encoded_left = model(left_input)
    encoded_right = model(right_input)

    L1_layer = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_left, encoded_right])

    prediction = Dense(1, activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)

    return siamese_net

input_shape = (128, 128, 3)
model = siamese_network(input_shape)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [3]:
# Veri çiftleri oluştur
def create_pairs(temiz, kirli):
    pairs = []
    labels = []
    for t_img in temiz:
        for k_img in kirli:
            pairs += [[t_img, k_img]]
            labels += [1]  # farklı oldukları için etiket 1

    for i in range(len(temiz)):
        for j in range(i + 1, len(temiz)):
            pairs += [[temiz[i], temiz[j]]]
            labels += [0]  # aynı oldukları için etiket 0

    for i in range(len(kirli)):
        for j in range(i + 1, len(kirli)):
            pairs += [[kirli[i], kirli[j]]]
            labels += [0]  # aynı oldukları için etiket 0

    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs(temiz_goruntuler, kirli_goruntuler)

# Verileri karıştır
indices = np.arange(len(labels))
np.random.shuffle(indices)
pairs = pairs[indices]
labels = labels[indices]

left_images = pairs[:, 0]
right_images = pairs[:, 1]

# Modeli eğitin
model.fit([left_images, right_images], labels, epochs=20, batch_size=16)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.5000 - loss: 0.6779
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6667 - loss: 0.6895
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.6896
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.5000 - loss: 0.6921
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.5000 - loss: 0.6887
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.6667 - loss: 0.6920
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.6933
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.3333 - loss: 0.6966
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.1667 - loss: 0.7005
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.8333 - loss: 0.6803
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.5000 - loss: 0.6874
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.0000e+00 - loss: 0.6979
Epoch 13/

# test

In [10]:
import cv2
import numpy as np

def load_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))  # Modelin eğitim boyutuna uygun olarak yeniden boyutlandırma
    img = np.array(img, dtype=np.float32)
    img /= 255.0  # Normalizasyon
    return img

# Test için kullanılacak resim
test_image = load_image('/content/clean/mutfak1.jpg')  # Temiz veya kirli resim dosyasının yolu


In [11]:

def predict_cleanliness(model, image):
    # Tek görüntü girişine uygun hale getirme
    image = np.expand_dims(image, axis=0)
    # Tahmin yapma
    prediction = model.predict([image, image])  # Kendisiyle karşılaştırarak tahmin yapıyoruz
    return prediction[0][0]

# Görüntünün temiz mi kirli mi olduğunu tahmin etme
prediction_score = predict_cleanliness(model, test_image)
print(f'Tahmin Skoru: {prediction_score}')

if prediction_score < 0.5:  # 0.5'ten düşükse resim kirli
    print("Resim kirli.")
else:  # 0.5'ten yüksekse resim temiz
    print("Resim temiz.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
Tahmin Skoru: 0.4996390640735626
Resim kirli.


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model

# Siamese Network modelini oluştur
def build_siamese_model(input_shape):
    input = Input(input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    model = Model(input, x)
    return model

def siamese_network(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    model = build_siamese_model(input_shape)
    encoded_left = model(left_input)
    encoded_right = model(right_input)
    L1_layer = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_left, encoded_right])
    prediction = Dense(1, activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input, right_input], outputs=prediction)
    return siamese_net

input_shape = (128, 128, 3)
model = siamese_network(input_shape)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Veri setini hazırlayın
def load_data():
    clean_images = []  # Temiz resimler için liste
    dirty_images = []  # Kirli resimler için liste

    # Temiz resimleri yükle
    for filename in os.listdir('temiz'):
        img = cv2.imread(os.path.join('temiz', filename))
        img = cv2.resize(img, (128, 128))
        clean_images.append(img)

    # Kirli resimleri yükle
    for filename in os.listdir('kirli'):
        img = cv2.imread(os.path.join('kirli', filename))
        img = cv2.resize(img, (128, 128))
        dirty_images.append(img)

    return np.array(clean_images), np.array(dirty_images)

clean_images, dirty_images = load_data()

# Eğitim ve doğrulama için çiftler oluştur
def create_pairs(clean_images, dirty_images):
    pairs = []
    labels = []
    num_images = min(len(clean_images), len(dirty_images))

    for i in range(num_images):
        pairs.append([clean_images[i], dirty_images[i]])
        labels.append(0)  # Farklı

        pairs.append([clean_images[i], clean_images[i]])
        labels.append(1)  # Aynı

    return np.array(pairs), np.array(labels)

pairs, labels = create_pairs(clean_images, dirty_images)

# Modeli eğit
model.fit([pairs[:, 0], pairs[:, 1]], labels, epochs=20, batch_size=16)

# Modeli kullanarak iki resmi karşılaştır
def compare_images(img1, img2):
    img1 = cv2.resize(img1, (128, 128))
    img2 = cv2.resize(img2, (128, 128))
    similarity = model.predict([np.array([img1]), np.array([img2])])
    return similarity[0][0]

# Örnek karşılaştırma
test_img1 = cv2.imread('temiz/test.jpg')
test_img2 = cv2.imread('kirli/test.jpg')
similarity_score = compare_images(test_img1, test_img2)
print(f'Benzerlik Skoru: {similarity_score}')


# new


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
import numpy as np

# Siamese ağda kullanılacak temel CNN modeli
def create_base_network(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(128)(x)  # Öznitelik vektörü (embedding)

    return Model(inputs, outputs)

# Mesafe fonksiyonu: Euclidean mesafesi
def euclidean_distance(vects):
    x, y = vects
    return tf.sqrt(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True))

# Mesafe fonksiyonu için özel katman
def euclidean_distance_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

# Modelin eğitimi için contrastive loss fonksiyonu
def contrastive_loss(y_true, y_pred):
    margin = 1
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# Verilerin şekli (örnek: 128x128x3 boyutunda mutfak görselleri)
input_shape = (128, 128, 3)

# İkiz ağlar (aynı CNN yapısı her iki giriş görüntüsü için kullanılır)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# İki görüntüyü öznitelik vektörlerine dönüştürüyoruz
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Mesafe hesaplanıyor
distance = layers.Lambda(euclidean_distance,
                         output_shape=euclidean_distance_output_shape)([processed_a, processed_b])

# Model oluşturuluyor
model = Model([input_a, input_b], distance)

# Model derleniyor
model.compile(loss=contrastive_loss, optimizer='adam')

# Örnek veriler
# İkiz görüntü çiftleri (temiz mutfak ve kirli mutfak çiftleri)
# Bu kısımda kendi veri setinizi kullanmalısınız
X_train_a = np.random.random((1000, 128, 128, 3))  # 1000 adet temiz mutfak görseli
X_train_b = np.random.random((1000, 128, 128, 3))  # 1000 adet kirli mutfak görseli
y_train = np.random.randint(0, 2, 1000)  # Temiz (1) veya kirli (0)

# Modelin eğitilmesi
model.fit([X_train_a, X_train_b], y_train, batch_size=32, epochs=10)

# Test için örnek görüntü çiftleri
X_test_a = np.random.random((10, 128, 128, 3))
X_test_b = np.random.random((10, 128, 128, 3))

# Modelin çıktısı, iki görüntü arasındaki mesafeyi verir
predictions = model.predict([X_test_a, X_test_b])
print("Benzerlik skorları:", predictions)


In [14]:
!pip install tensorflow numpy

In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Verilerin konumları
clean_image_folder = '/content/clean'  # Temiz mutfak görüntülerinin olduğu klasör
dirty_image_folder = '/content/dirty'  # Kirli mutfak görüntülerinin olduğu klasör

# Görselleri yükleme fonksiyonu
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Temiz ve kirli mutfak görsellerini yükleme
clean_images = load_images_from_folder(clean_image_folder)
dirty_images = load_images_from_folder(dirty_image_folder)

# Etiketler (clean = 1, dirty = 0)
clean_labels = np.ones(len(clean_images))
dirty_labels = np.zeros(len(dirty_images))

# Verileri ve etiketleri birleştirip train/test seti oluşturma
X = np.concatenate([clean_images, dirty_images], axis=0)
y = np.concatenate([clean_labels, dirty_labels], axis=0)

# Train/test seti oluşturma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli beslemek için uygun formatta çiftler oluşturma (Siamese için)
def make_pairs(images, labels):
    pairs = []
    labels_pairs = []
    num_classes = len(np.unique(labels))

    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]

        # Aynı sınıftan bir görüntü ile çift
        idx2 = np.random.choice(np.where(labels == label)[0])
        positive_image = images[idx2]
        pairs += [[current_image, positive_image]]
        labels_pairs += [1]  # Benzer çift

        # Farklı sınıftan bir görüntü ile çift
        idx2 = np.random.choice(np.where(labels != label)[0])
        negative_image = images[idx2]
        pairs += [[current_image, negative_image]]
        labels_pairs += [0]  # Farklı çift

    return np.array(pairs), np.array(labels_pairs)

# Eğitim ve test verilerinden çiftler oluşturma
train_pairs, train_labels = make_pairs(X_train, y_train)
test_pairs, test_labels = make_pairs(X_test, y_test)

# Eğitim verilerini modele uygun hale getirme,

X_train_a = train_pairs[:, 0]  # İlk görüntü
X_train_b = train_pairs[:, 1]  # İkinci görüntü
y_train = train_labels

X_test_a = test_pairs[:, 0]
X_test_b = test_pairs[:, 1]
y_test = test_labels

# Verileri normalize etme
X_train_a = X_train_a / 255.0
X_train_b = X_train_b / 255.0
X_test_a = X_test_a / 255.0
X_test_b = X_test_b / 255.0


ValueError: 'a' cannot be empty unless no samples are taken

In [ ]:
# Modeli eğitme
model.fit([X_train_a, X_train_b], y_train, batch_size=32, epochs=10, validation_data=([X_test_a, X_test_b], y_test))

# Test için tahminler
predictions = model.predict([X_test_a, X_test_b])
print("Benzerlik skorları:", predictions)


In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Verilerin konumları
clean_image_folder = '/content/C'  # Temiz mutfak görüntülerinin olduğu klasör
dirty_image_folder = '/content/d'  # Kirli mutfak görüntülerinin olduğu klasör

# Görselleri yükleme fonksiyonu
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Temiz ve kirli mutfak görsellerini yükleme
clean_images = load_images_from_folder(clean_image_folder)
dirty_images = load_images_from_folder(dirty_image_folder)

# Etiketler (clean = 1, dirty = 0)
clean_labels = np.ones(len(clean_images))
dirty_labels = np.zeros(len(dirty_images))

# Verileri ve etiketleri birleştirip train/test seti oluşturma
X = np.concatenate([clean_images, dirty_images], axis=0)
y = np.concatenate([clean_labels, dirty_labels], axis=0)

# Train/test seti oluşturma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verileri normalleştirme
X_train = X_train / 255.0
X_test = X_test / 255.0
def make_pairs(images, labels):
    pairs = []
    labels_pairs = []
    unique_labels = np.unique(labels)

    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]

        # Aynı sınıftan bir görüntü ile çift
        idx2 = np.random.choice(np.where(labels == label)[0])
        positive_image = images[idx2]
        pairs += [[current_image, positive_image]]
        labels_pairs += [1]  # Benzer çift

        # Farklı sınıftan bir görüntü ile çift (Eğer farklı sınıftan veri varsa)
        other_labels = unique_labels[unique_labels != label]
        if len(other_labels) > 0:  # Farklı sınıf varsa devam et
            negative_label = np.random.choice(other_labels)
            idx2 = np.random.choice(np.where(labels == negative_label)[0])
            negative_image = images[idx2]
            pairs += [[current_image, negative_image]]
            labels_pairs += [0]  # Farklı çift

    return np.array(pairs), np.array(labels_pairs)


# Eğitim ve test verilerinden çiftler oluşturma
train_pairs, train_labels = make_pairs(X_train, y_train)
test_pairs, test_labels = make_pairs(X_test, y_test)

X_train_a = train_pairs[:, 0]
X_train_b = train_pairs[:, 1]
X_test_a = test_pairs[:, 0]
X_test_b = test_pairs[:, 1]

# Modeli normalize etme
X_train_a = X_train_a / 255.0
X_train_b = X_train_b / 255.0
X_test_a = X_test_a / 255.0
X_test_b = X_test_b / 255.0

# Siamese network'in temelini oluşturan CNN modeli
def create_base_model(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    return Model(input, x)

# İki görüntü arasındaki mesafeyi hesaplayan fonksiyon
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Modeli oluşturma
input_shape = (128, 128, 3)  # Görüntü boyutları
base_model = create_base_model(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

feats_a = base_model(input_a)
feats_b = base_model(input_b)

distance = Lambda(euclidean_distance)([feats_a, feats_b])

model = Model(inputs=[input_a, input_b], outputs=distance)

# Modeli derleme
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Modeli eğitme
model.fit([X_train_a, X_train_b], train_labels, batch_size=128, epochs=100, validation_data=([X_test_a, X_test_b], test_labels))

# Test için tahminler
predictions = model.predict([X_test_a, X_test_b])
print("Benzerlik skorları:", predictions)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.5000 - loss: 4.0273 - val_accuracy: 0.5000 - val_loss: 4.0298
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 3.8003 - val_accuracy: 0.5000 - val_loss: 4.0302
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 2.9622 - val_accuracy: 0.5000 - val_loss: 4.0320
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.5000 - loss: 2.2702 - val_accuracy: 0.5000 - val_loss: 4.0371
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.5000 - loss: 1.7231 - val_accuracy: 0.5000 - val_loss: 4.0475
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.5000 - loss: 1.3719 - val_accuracy: 0.5000 - val_loss: 4.0674
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.5000 - loss: 1.1293 - val_accuracy: 0.5000 - val_loss: 4.1347
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.7500 - loss: 0.9042 - val_accuracy: 0.5000 - val_loss:

In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Verilerin konumları
clean_image_folder = '/content/C'  # Temiz mutfak görüntülerinin olduğu klasör
dirty_image_folder = '/content/d'  # Kirli mutfak görüntülerinin olduğu klasör

# Görselleri yükleme fonksiyonu
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

# Temiz ve kirli mutfak görsellerini yükleme
clean_images = load_images_from_folder(clean_image_folder)
dirty_images = load_images_from_folder(dirty_image_folder)

# Etiketler (clean = 1, dirty = 0)
clean_labels = np.ones(len(clean_images))
dirty_labels = np.zeros(len(dirty_images))

# Verileri ve etiketleri birleştirip train/test seti oluşturma
X = np.concatenate([clean_images, dirty_images], axis=0)
y = np.concatenate([clean_labels, dirty_labels], axis=0)

# Train/test seti oluşturma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verileri normalleştirme
X_train = X_train / 255.0
X_test = X_test / 255.0
def make_pairs(images, labels):
    pairs = []
    labels_pairs = []
    unique_labels = np.unique(labels)

    for idx1 in range(len(images)):
        current_image = images[idx1]
        label = labels[idx1]

        # Aynı sınıftan bir görüntü ile çift
        idx2 = np.random.choice(np.where(labels == label)[0])
        positive_image = images[idx2]
        pairs += [[current_image, positive_image]]
        labels_pairs += [1]  # Benzer çift

        # Farklı sınıftan bir görüntü ile çift (Eğer farklı sınıftan veri varsa)
        other_labels = unique_labels[unique_labels != label]
        if len(other_labels) > 0:  # Farklı sınıf varsa devam et
            negative_label = np.random.choice(other_labels)
            idx2 = np.random.choice(np.where(labels == negative_label)[0])
            negative_image = images[idx2]
            pairs += [[current_image, negative_image]]
            labels_pairs += [0]  # Farklı çift

    return np.array(pairs), np.array(labels_pairs)


# Eğitim ve test verilerinden çiftler oluşturma
train_pairs, train_labels = make_pairs(X_train, y_train)
test_pairs, test_labels = make_pairs(X_test, y_test)

X_train_a = train_pairs[:, 0]
X_train_b = train_pairs[:, 1]
X_test_a = test_pairs[:, 0]
X_test_b = test_pairs[:, 1]

# Modeli normalize etme
X_train_a = X_train_a / 255.0
X_train_b = X_train_b / 255.0
X_test_a = X_test_a / 255.0
X_test_b = X_test_b / 255.0

# Siamese network'in temelini oluşturan CNN modeli
def create_base_model(input_shape):
    input = Input(shape=input_shape)
    x = Conv2D(64, (10, 10), activation='relu')(input)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (7, 7), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(128, (4, 4), activation='relu')(x)
    x = MaxPooling2D()(x)
    x = Conv2D(256, (4, 4), activation='relu')(x)
    x = Flatten()(x)
    x = Dense(4096, activation='sigmoid')(x)
    return Model(input, x)

# İki görüntü arasındaki mesafeyi hesaplayan fonksiyon
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sum_squared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_squared, K.epsilon()))

# Modeli oluşturma
input_shape = (128, 128, 3)  # Görüntü boyutları
base_model = create_base_model(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

feats_a = base_model(input_a)
feats_b = base_model(input_b)

distance = Lambda(euclidean_distance)([feats_a, feats_b])

model2 = Model(inputs=[input_a, input_b], outputs=distance)

# Modeli derleme
model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Modeli eğitme
model2.fit([X_train_a, X_train_b], train_labels, batch_size=32, epochs=100, validation_data=([X_test_a, X_test_b], test_labels))

# Test için tahminler
predictions2 = model2.predict([X_test_a, X_test_b])
print("Benzerlik skorları:", predictions2)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: 4.0297
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - accuracy: 0.5000 - loss: 4.0297 - val_accuracy: 0.5000 - val_loss: